In [1]:
import pandas as pd
import numpy as np
import json
import os
from IPython.display import display

import scipy, scipy.io

import matplotlib
from matplotlib import *
from matplotlib import pyplot as plt
import itertools
from mpl_toolkits.axes_grid1 import make_axes_locatable
# pretty charting
import seaborn as sns
sns.set_palette('muted')
sns.set_style('darkgrid')
from natsort import natsorted, ns

%matplotlib inline

## sklearn imports
import sklearn.linear_model
import sklearn.cross_validation
import sklearn.tree
import sklearn.ensemble
import sklearn.preprocessing
import sklearn.feature_selection
import sklearn.pipeline
import sklearn.grid_search

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
def loadmat(filename):
    '''
    this function should be called instead of direct spio.loadmat
    as it cures the problem of not properly recovering python dictionaries
    from mat files. It calls the function check keys to cure all entries
    which are still mat-objects
    '''
    data = scipy.io.loadmat(filename, struct_as_record=False, squeeze_me=True)
    return _check_keys(data)

def _check_keys(dict):
    '''
    checks if entries in dictionary are mat-objects. If yes
    todict is called to change them to nested dictionaries
    '''
    for key in dict:
        if isinstance(dict[key], scipy.io.matlab.mio5_params.mat_struct):
            dict[key] = _todict(dict[key])
    return dict        

def _todict(matobj):
    '''
    A recursive function which constructs from matobjects nested dictionaries
    '''
    dict = {}
    for strg in matobj._fieldnames:
        elem = matobj.__dict__[strg]
        if isinstance(elem, scipy.io.matlab.mio5_params.mat_struct):
            dict[strg] = _todict(elem)
        elif isinstance(elem,np.ndarray):
            dict[strg] = _tolist(elem)
        else:
            dict[strg] = elem
    return dict

def _tolist(ndarray):
    '''
    A recursive function which constructs lists from cellarrays 
    (which are loaded as numpy ndarrays), recursing into the elements
    if they contain matobjects.
    '''
    elem_list = []            
    for sub_elem in ndarray:
        if isinstance(sub_elem, scipy.io.matlab.mio5_params.mat_struct):
            elem_list.append(_todict(sub_elem))
        elif isinstance(sub_elem,np.ndarray):
            elem_list.append(_tolist(sub_elem))
        else:
            elem_list.append(sub_elem)
    return elem_list

In [7]:
dataFile = '../Monkey X/clusters/pearsonRSpectralClustered_allconditions'
data = loadmat(dataFile)


### Visualize Push

In [30]:
# get conditions, trials, and clusters
conditionData = data['cluster_struct']['push']
trials = conditionData.keys()
clusters = conditionData[trials[0]].keys()
clusters.remove('info')

info = conditionData[trials[0]]['info']
print info.keys()
for cluster in clusters: # loop through each cluster
    print conditionData[trials[0]]['info']['neuronsInCluster']
#     print conditionData[trials[0]]['info']['neuronsInCluster'].keys()
    print np.asarray(conditionData[trials[0]][cluster]).size
    print np.asarray(conditionData[trials[3]][cluster]).size
print clusters

['trialnum', 'close_time', 'instruction_time', 'neuronsInCluster']
[u'sig001b.mat', u'sig001c.mat', u'sig003b.mat', u'sig003c.mat', u'sig004a.mat', u'sig004b.mat', u'sig005a.mat', u'sig005b.mat', u'sig005c.mat', u'sig007a.mat', u'sig007b.mat', u'sig007c.mat', u'sig007d.mat', u'sig008c.mat', u'sig009a.mat', u'sig009b.mat', u'sig012b.mat', u'sig014a.mat', u'sig014d.mat', u'sig015a.mat', u'sig015b.mat', u'sig020a.mat', u'sig021a.mat', u'sig027a.mat', u'sig028a.mat', u'sig033a.mat', u'sig033c.mat', u'sig033d.mat', u'sig034c.mat', u'sig034d.mat', u'sig035b.mat', u'sig036d.mat', u'sig037a.mat', u'sig037b.mat', u'sig038b.mat', u'sig038c.mat', u'sig038d.mat', u'sig039b.mat', u'sig040a.mat', u'sig040b.mat', u'sig041b.mat', u'sig041c.mat', u'sig041d.mat', u'sig042b.mat', u'sig043a.mat', u'sig043b.mat', u'sig046c.mat', u'sig047b.mat', u'sig047d.mat', u'sig052a.mat', u'sig054a.mat', u'sig056a.mat', u'sig058a.mat', u'sig060a.mat', u'sig060c.mat', u'sig070b.mat', u'sig071b.mat', u'sig071c.mat', u'si